In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import neurokit2 as nk
from sklearn import __version__ as sklearn_version

print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print(f"SciPy: {sp.__version__}")
print(f"NeuroKit2: {nk.__version__}")
print(f"Scikit-learn: {sklearn_version}")

Pandas: 2.0.3
NumPy: 1.24.4
SciPy: 1.10.1
NeuroKit2: 0.2.10
Scikit-learn: 1.3.2


In [11]:
# 单元格 1: Markdown 说明
# PTT预测血压数据分析
#
# 本笔记本用于读取并分析HUB文件夹中的PPG数据和Biopac文件夹中的血压数据，检查每列的统计信息。

# 单元格 2: 导入必要的库
import pandas as pd
import numpy as np
import os
from scipy import stats

# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# 单元格 3: 定义数据路径
# 定义数据路径（请根据实际路径修改）
hub_folder = '/root/PI_Lab/00017/1/HUB'  # 替换为您的HUB文件夹路径
Biopac_folder = '/root/PI_Lab/00017/1/Biopac'  

# 列出HUB文件夹中的所有CSV文件
hub_files = [f for f in os.listdir(hub_folder) if f.endswith('.csv')]
print("HUB文件夹中的CSV文件:", hub_files)

# 列出Biopac文件夹中的所有CSV文件
Biopac_files = [f for f in os.listdir(Biopac_folder) if f.endswith('.csv')]
print("Biopac文件夹中的CSV文件:", Biopac_files)


HUB文件夹中的CSV文件: ['sensor4.csv', 'sensor3.csv', 'sensor5.csv', 'sensor2.csv']
Biopac文件夹中的CSV文件: ['diastolic_bp-1.csv', 'rsp-1.csv', 'systemic_vascular_resistance-1.csv', 'hr-1.csv', 'cardiac_output-1.csv', 'bp-1.csv', 'systolic_bp-1.csv', 'mean_bp-1.csv', 'cardiac_index-1.csv']


In [14]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [15]:
# 单元格 4: 查看每个文件的列名
# 函数：读取并显示文件列名
def show_columns(file_path, folder_type='hub'):
    try:
        # 读取CSV文件
        df = pd.read_csv(file_path)
        print(f"\n{folder_type.upper()} 文件: {os.path.basename(file_path)}")
        print("列名:", df.columns.tolist())
        print("前2行数据预览:")
        print(df.head(2))
    except Exception as e:
        print(f"读取文件 {file_path} 出错: {e}")

# 分析HUB文件夹中的每个文件
for file in hub_files:
    file_path = os.path.join(hub_folder, file)
    show_columns(file_path, folder_type='hub')

# 分析Biopac文件夹中的每个文件
for file in Biopac_files:
    file_path = os.path.join(Biopac_folder, file)
    show_columns(file_path, folder_type='biopac')


HUB 文件: sensor4.csv
列名: ['timestamp', 'red', 'ir', 'green', 'ax', 'ay', 'az', 'rx', 'ry', 'rz', 'mx', 'my', 'mz', 'temp', 'time']
前2行数据预览:
      timestamp      red        ir   green    ax    ay    az    rx   ry    rz    mx     my    mz   temp      time
0  1.750838e+09  72802.0  124688.0  3427.0 -0.59  9.52  1.85 -3.35 -1.1 -3.72 -7.35  47.55 -4.95  34.48  218419.0
1  1.750838e+09  72802.0  124688.0  3427.0 -0.59  9.52  1.85 -3.35 -1.1 -3.72 -7.35  47.55 -4.95  34.48  218428.0

HUB 文件: sensor3.csv
列名: ['timestamp', 'red', 'ir', 'green', 'ax', 'ay', 'az', 'rx', 'ry', 'rz', 'mx', 'my', 'mz', 'temp', 'time']
前2行数据预览:
      timestamp       red        ir   green    ax    ay   az   rx     ry    rz    mx   my     mz   temp      time
0  1.750838e+09  119489.0  137421.0  5585.0 -9.48  3.28  0.2 -7.8 -21.65  0.06  31.5  7.5 -18.75  34.55  218419.0
1  1.750838e+09  119489.0  137421.0  5585.0 -9.48  3.28  0.2 -7.8 -21.65  0.06  31.5  7.5 -18.75  34.55  218428.0

HUB 文件: sensor5.csv
列名: ['timestamp